# Part 1 of the assignment

## Import all Necessary libraries


In [ ]:
import numpy as np
import pandas as pd

import json
from pandas.io.json import json_normalize 

import matplotlib as plt
%matplotlib inline

#from yellowbrick.cluster import KElbowVisualizer

from sklearn.cluster import KMeans
print("Libraries imported")

### Get the Wiki Data and view the first 5 entries 

In [ ]:
url_wiki = "https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
df_list = pd.read_html(url_wiki)
df_list[0].head()

### Create a dataframe and assign the column names which are present as first row of the table/dataframe

In [ ]:
canada_data = pd.DataFrame(df_list[0])
header = canada_data.iloc[0]
canada_data = canada_data.iloc[1:]
canada_data.columns = header
canada_data.head()

In [ ]:
#Group by Borough to see which borough is "NOT ASSIGNED" 
canada_data.groupby("Borough").count()

In [ ]:
## Remove borough NOT ASSIGNED
canada_data = canada_data[canada_data["Borough"] != "Not assigned"].reset_index(drop = True)
canada_data.groupby("Borough").count()

In [ ]:
## Replace not assigned Neighbourhood values with the Borough value
for i in range(canada_data.shape[0]):
    if canada_data["Neighbourhood"][i] == "Not assigned":
        canada_data["Neighborhood"][i] == canada_data["Borough"][i]
        
    

In [ ]:
# Combine neighbourhoods with same postal code
canada_data["Neighbourhood"] = canada_data.groupby(by = ["Postal Code" , "Borough"])['Neighbourhood'].transform(lambda x: ','.join(x))


In [ ]:
canada_data.shape

# Part 2 of the assignment.
## Reading the geospatial coordinates and merging them with the canada data file to obtain the required dataframe

In [ ]:
geo_coordinates = pd.read_csv("Geospatial_Coordinates.csv" , header=0)
geo_coordinates.head()

In [ ]:
data = pd.merge(canada_data,geo_coordinates,on = "Postal Code" , how = "left")

In [ ]:
data.head()

In [ ]:
data.shape

In [ ]:

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
import numpy as np

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes
import folium # map rendering library

In [ ]:
CLIENT_ID = "S0TZRLOEGDM0AC4IF1EARQ5AOIMHS2C3PJK3BMJQO353L3TE"
CLIENT_SECRET = "PAUU3ENZ2KFN0Q5VJTV0T15WL34YT24V5NT14NKVAJUSAGVT"
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
LIMIT = 100
radius = 500
toronto_venues = getNearbyVenues(names=data['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

In [ ]:
toronto_venues.head()


In [ ]:
toronto_venues.groupby('Neighbourhood').count()


In [ ]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

In [ ]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighbourhood'] = toronto_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

toronto_onehot.head()

In [ ]:
toronto_onehot.shape

In [ ]:
toronto_grouped = toronto_onehot.groupby('Neighbourhood').mean().reset_index()
toronto_grouped

In [ ]:
toronto_grouped.shape

In [ ]:
num_top_venues = 5

for hood in toronto_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = toronto_grouped['Neighbourhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
kclusters = 5

toronto_grouped_clustering = toronto_grouped.drop('Neighbourhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

In [ ]:
# add clustering labels
#del neighbourhoods_venues_sorted
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighbourhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighbourhood'), on='Neighbourhood',how='right')

toronto_merged.head() # check the last columns!